## 1 A

In [28]:
import sys, requests, re, random, os

url, auth = 'http://131.155.21.174:8084', ('', 'ilovecrypto')
sid, token = 20221886, '6950e84b50d4e62cc6be6010a8333a5c2ad46f7f'

req = requests.Session()
req.cookies['token'] = token

r = req.get(url+'/', auth=auth)
target = bytes.fromhex(re.search('target_hash = ([0-9a-f]+)', r.text).groups()[0])
target_hash = "8afc96c8d0c34b744263d5c5c609d616"

def hash(x):
    r = req.post(url+'/hash', auth=auth, data={'data': x.hex()})
    return bytes.fromhex(r.text)

def multi_unhash(xs):
    r = req.post(url+'/multi_unhash', auth=auth, data={'hashes': [x.hex() for x in xs]})
    r = r.text.strip().split()
    if r[1] == 'None': 
        return int(r[0]), None  # bad query
    return int(r[0]), bytes.fromhex(r[1])

def preimage(x):  # use this to submit your preimage
    r = req.post(url+'/validate_preimage', auth=auth, data={'data': x.hex()})
    print('\x1b[32m{}\x1b[0m'.format(r.text.strip()))

################################################################
print(f'target hash value: {target}')
# print('target hash value: {}'.format(target.hex()))

# let's compute a few hash values for fun

def generate_hashes():
    xs = []
    for _ in range(24):
        x = os.urandom(random.randrange(5,20))
        message = x.hex()
        message_hash = hash(x)
        xs.append(message_hash)
    return xs

target hash value: b'\x8a\xfc\x96\xc8\xd0\xc3KtBc\xd5\xc5\xc6\t\xd6\x16'


In [38]:
def solve():
    print(f"target hash {target_hash}")
    i = 0
    while True:
        i += 1
        xs = generate_hashes()
        xs.insert(12, target)
        preimage = multi_unhash(xs)
        preimage_hash = hash(preimage[1]).hex()
        print(f"preimage hash {i} = {preimage_hash}")
        if (preimage_hash == target2):
            print(f"preimage {preimage[1].hex()}")
            print(f"preimage hash {preimage_hash}")
            return preimage[1].hex()

In [39]:
sol = solve()
sol

target hash 8afc96c8d0c34b744263d5c5c609d616
preimage hash 1 = 7fbc993a71743a1adb220568f287979c
preimage hash 2 = 8e538c10ad4a8b3ebe7c525747b779d4
preimage hash 3 = 8b0270d3e9c455c3725432bac3a371d5
preimage hash 4 = 8afc96c8d0c34b744263d5c5c609d616
preimage 48656c6c6f2c2032303232313838362e205468697320697320796f757220707265696d6167652e20536f6d652072616e646f6d6e6573733a203337326166663438313239393134323837636536643036666539386463636339
preimage hash 8afc96c8d0c34b744263d5c5c609d616


'48656c6c6f2c2032303232313838362e205468697320697320796f757220707265696d6167652e20536f6d652072616e646f6d6e6573733a203337326166663438313239393134323837636536643036666539386463636339'

## 1 B

In [40]:
N = 47641986688040193867377071289707038226486095080434488478557235920260051619212976109300257901646278642780852000629457258208885956988467583153162929158543252389566040916909256552175444520709560832795397422059532153740131565807964720296262734778654915668031798555272566354293548000081225499135036903655561872481
g = 1973562372334739436117157396994411802795036014353454350182655881706341376046726071153435464248164251020750758985477740519314050545345059627577624541367681592362875422542356588779142865255880725220775657732229921437703762688429345758213803939418815981097096148970297596554980802312428187930688849077340601082
mod(g^2, N)

42

In [41]:
import sys, requests, re, random

url, auth = 'http://131.155.21.174:8184', ('', 'ilovecrypto')
sid, token = 20221886, '7dea095feb8c9c0fa5d12d553ab721de4577957d'

req = requests.Session()
req.cookies['token'] = token

r = req.get(url+'/', auth=auth)
n = int(re.findall('n = ([0-9]+)', r.text)[0])

def sqrt(x):
    r = req.post(url+'/sqrt', auth=auth, data={'value': x})
    if r.text.strip() == 'None': return None  # not a square
    return int(r.text)

def factorization(p, q):  # use this to submit your factorization
    assert type(p) == type(q) == int
    r = req.post(url+'/validate_factorization', auth=auth, data={'p': '{:d}'.format(p), 'q': '{:d}'.format(q)})
    print('\x1b[32m{}\x1b[0m'.format(r.text.strip()))

################################################################

print('n: {}'.format(n))

# let's compute a few square roots for fun

for _ in range(10):
    x = random.randrange(10**10)
    print('value: {}'.format(x), end = ' ~> ')
    print('sqrt: {}'.format(sqrt(x)))

n: 47641986688040193867377071289707038226486095080434488478557235920260051619212976109300257901646278642780852000629457258208885956988467583153162929158543252389566040916909256552175444520709560832795397422059532153740131565807964720296262734778654915668031798555272566354293548000081225499135036903655561872481
value: 8537473694 ~> sqrt: None
value: 6173092323 ~> sqrt: None
value: 2451026709 ~> sqrt: None
value: 5227271663 ~> sqrt: None
value: 6390925781 ~> sqrt: None
value: 5565618969 ~> sqrt: None
value: 5857686769 ~> sqrt: None
value: 5761937298 ~> sqrt: None
value: 6203830910 ~> sqrt: None
value: 5891617749 ~> sqrt: 5557440900192710505772330666355067727623638350204603051036177120319139034026399354749683544762925945562860627282480744059265789445799138783073460768714874089363354183222744682665812423422084235845671557144381488961589998109256701797457058349386700750785047748570591015164099313567659713316122964377601851510


In [58]:
def solve():
    # https://crypto.stackexchange.com/questions/34061/factoring-large-n-given-oracle-to-find-square-roots-modulo-n
    # https://math.stackexchange.com/questions/633160/modular-arithmetic-find-the-square-root
    while True:
        x = random.randint(0, N-1)
        x2 = int(mod(x^2, N))
        # print('value: {}'.format(x2), end = ' -> ')
        y = sqrt(x2)
        # print('sqrt: {}'.format(y))
        if y:
            y2 = int(mod(y^2, N))
            # print(f"x2 {x2} == {y2} y2")
            sol = gcd(x + y, N)    
            if sol != 1 and sol != N:
                return sol

In [59]:
sol = solve()
p = sol
q = N / p
print(f"Solution! p: {p}, q: {q}")

Solution! p: 13296306367833070932025183798119196681658330744582826048552378042206097171995410042424539861419819383823964657670268624773688144521607820415276326211219113, q: 3583099348800919363874090572250302828397954662694388937368140488569787142280059937122900813045905801462205862616043865033586780383720026774964829885158137


In [61]:
N == p * q

True

In [62]:
p

13296306367833070932025183798119196681658330744582826048552378042206097171995410042424539861419819383823964657670268624773688144521607820415276326211219113

In [63]:
q

3583099348800919363874090572250302828397954662694388937368140488569787142280059937122900813045905801462205862616043865033586780383720026774964829885158137